### 抓取微博发言

- 作者：DGC'Idea
- 版本：V0.3   
- 更新日期：2018年4月9日

----------

- 研究目的：获取关注微博博主的发言；
- 研究方法：从微博手机主页爬取所有发言。

In [1]:
#-*- coding: utf-8 -*-

# 基本
import numpy as np
import pandas as pd

# 系统
import os

# 网络
import urllib.request
import requests
import json
import re

# 日期、时间
import time
from datetime import timedelta,date,datetime

# 网页解析
from bs4 import BeautifulSoup

# 自定义
import sys
sys.path.append('../DM')
sys.path.append('../TL')

from dm import *
from tl import exists_file


# 数据目录
_data_path='../Data/'
# 数据引擎
_data=Csv(_data_path)


#设置代理IP
proxy_addr="122.241.72.191:808"

#性别
Gender={'m':'男','w':'女'}

#性别
Accreditation={'True':'是','False':'否'}

#使用代理访问页面
def use_proxy(url,proxy_addr):
    req=urllib.request.Request(url)
    req.add_header("User-Agent","Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36\
                   (KHTML, like Gecko) Chrome/49.0.2623.221 Safari/537.36 SE 2.X MetaSr 1.0")
    proxy=urllib.request.ProxyHandler({'http':proxy_addr})
    opener=urllib.request.build_opener(proxy,urllib.request.HTTPHandler)
    urllib.request.install_opener(opener)
    data=urllib.request.urlopen(req).read().decode('utf-8','ignore')
    return data



#获取微博账号的用户基本信息，如：微博昵称、微博地址、微博头像、关注人数、粉丝数、性别、等级等
def get_weibo_userInfo(id):
    #url地址
    url='https://m.weibo.cn/api/container/getIndex?type=uid&value='+id
    #使用代理访问页面
    data=use_proxy(url,proxy_addr)
    #解析数据
    content=json.loads(data).get('data')
    #头像地址
    image_url=content.get('userInfo').get('profile_image_url')
    #说明
    description=content.get('userInfo').get('description')
    #主页地址
    profile_url=content.get('userInfo').get('profile_url')
    #是否认证
    verified=content.get('userInfo').get('verified')
    #关注数
    follow_count=content.get('userInfo').get('follow_count')
    #昵称
    name=content.get('userInfo').get('screen_name')
    #粉丝数
    followers_count=content.get('userInfo').get('followers_count')
    #性别
    gender=content.get('userInfo').get('gender')
    #等级
    urank=content.get('userInfo').get('urank')
    statuses_count=content.get('userInfo').get('statuses_count')
    #显示信息
    return(
          name,
          Gender[gender],
          profile_url,
          Accreditation[str(verified)],
          urank, 
          follow_count,
          followers_count,
          statuses_count,
          description,    
         )


#获取微博主页的containerid，爬取微博内容时需要此id
def get_containerid(url):
#     data=use_proxy(url,proxy_addr)
    #直接请求模式
    data=requests.get(url).text 
    content=json.loads(data).get('data')
    for data in content.get('tabsInfo').get('tabs'):
        if(data.get('tab_type')=='weibo'):
            containerid=data.get('containerid')
    return containerid


#获取微博的更进一步信息：确切时间、完整正文
def get_weibo_text(url):
    #代理模式
#     data=use_proxy(url,proxy_addr)
    #直接请求模式
    data=requests.get(url).text 
    #剥离不相关信息
    data=data.split('var $render_data = [')[1].split('][0] ')[0]
    #取得json信息
    content=json.loads(data).get('status')
    #微博正文
    text=content['text']
    #完整时间
    created_at=content['created_at']
    #转换时间格式（原格式比较特别）
    created_at=str(datetime.strptime(created_at.replace('+0800',''),'%a %b %d %H:%M:%S %Y'))    
    #返回正文和时间
    return text,created_at
    
    
#获取微博内容：正文、详情页地址、点赞数、评论数、转发数等
def get_weibo_data(user_id,start_date=None,First=False):
    #页数计数器
    page=0
    #微博计数器
    count=0
    #数据表
    data_list=[]
    #微博名
    name=id_all_list[user_id]
    #退出标记
    loop=True
    #url（微博用户信息网址）
    url='https://m.weibo.cn/api/container/getIndex?type=uid&value='+user_id
    #获得完整id
    containerid=get_containerid(url)
    #页面循环
    while loop:
        #开启错误抓取
        try:
            # 页面数累计            
            page+=1    
            weibo_url='https://m.weibo.cn/api/container/getIndex?type=uid&value='\
                       +user_id+'&containerid='+containerid+'&page='+str(page)
  
            # 代理模式
#             data=use_proxy(weibo_url,proxy_addr) 
            #直接请求模式
            data=requests.get(weibo_url).text 
            # 取得josn数据
            content=json.loads(data).get('data')
            # 取得微博信息
            cards=content.get('cards')
            
            # 判断是否为空数据
            if(len(cards)==0):
                #退出循环
                break   
                
            #遍历当前页面所有微博  
            for card in cards:
                card_type=card.get('card_type')
                
                # 非微博发言
                if(card_type!=9):
                    continue
                    
                # 微博内容    
                mblog=card.get('mblog')
                
                #去除置顶微博
                if len(mblog.get('edit_at',''))!=0 :
                    continue
                    
                #微博实际网址    
                scheme=card.get('scheme')  
                #微博实际正文、发布时间    
                text,created_at=get_weibo_text(scheme)
                
                if created_at<=start_date:
                    loop=False
                    break
                    
                #微博数累计 
                count+=1
                print("\r"+"正在获取：%s %s 的微博，已获取 %s 条"%(name, created_at, count),end="")
                #添加到数据列表
                data_list.append([created_at,text,
                                mblog.get('attitudes_count'),mblog.get('comments_count'),mblog.get('reposts_count'),
                                scheme,False,
                                ])
            #延时
            time.sleep(1.0)
        #错误处理
        except Exception as e:
            print('错误：%s'%(e),end='\r')
            if '418' in str(e):
                #遇到418错误
                time.sleep(3.0)
                pass
            else:   
                #其它错误，延时后继续
                time.sleep(1.0)
                pass   
            
    if len(data_list)!=0:
        #数据表整理                
        columns_list=['created_at','text','attitudes_count','comments_count','reposts_count','scheme','sended']
        df=pd.DataFrame(data=data_list,columns=columns_list) 
        #返回数据
        return df
    else:
        return None
    
#增量更新
def update_weibo_data(id_list): 
    #遍历所有id
    for user_id in id_list:
        #微博名
        name=id_all_list[user_id] 
        df=pd.DataFrame()

        #判断文件是否存存在
        if exists_weibo_file(user_id):    #+'test'        
            #增量更新         
            #从cvs文件中导入数据
            df=read_weibo_data(user_id)
 
            #获得增量开始日期
            update_date=df['created_at'].iloc[0]  
        else:
            #初次计算
            update_date=None
            
        # 获取数据    
        update_df=get_weibo_data(user_id,update_date) 
        
        # 判断是否获取到数据
        if not update_df is None:
            print('共计更新：%s %s 条微博'%(name,len(update_df)))
            #拼接新旧数据，并重置索引
            df=pd.concat([update_df,df],ignore_index=True) 
            #return update_df
            #保存数据到cvs 
            save_weibo_data(user_id,df)
        
#检测文件      
def exists_weibo_file(user_id,data_root=_data_path):
    #cvs文件名
    data_file='%s%s.csv'%(data_root,'msg_weibo_'+user_id)
    return exists_file(data_file)


#读取数据        
def read_weibo_data(user_id):
    #文件名
    data_file='msg_weibo_'+user_id
    #从cvs文件恢复数据
    df=_data.read(data_file)
    
    return df

#保存数据        
def save_weibo_data(user_id,df):
    #文件名
    data_file='msg_weibo_'+user_id
    #保存到cvs文件
    _data.save(data_file,df,append=False)
    

#id列表
id_all_list={
    '5687069307':'ETF拯救世界',
#     '1669616825':'程老湿爱吐槽',
#     '1254381361':'闲来一坐'
        }

print(' 已准备好...')   

数据引擎：运行于研究
 已准备好...


## 1.更新数据

In [2]:
#更新数据
df=update_weibo_data(id_all_list)
df

正在获取：ETF拯救世界 2019-07-26 15:59:53 的微博，已获取 580 条共计更新：ETF拯救世界 580 条微博


## 2.设置当前ID

In [3]:
#获取当前微博ID信息
# get_weibo_userInfo(id_all_list.keys().tolist()[1])
user_id='5687069307'
(user_name,
gender,
profile_url,
verified,
urank, 
follow_count,
followers_count,
statuses_count,
description)=get_weibo_userInfo(user_id)
print(
      " 微博昵称："+user_name+"\n"+
      " 性　　别："+gender+"\n"+
      " 主页地址："+profile_url+"\n"+
      " 是否认证："+verified+"\n"+
      " 等　　级："+str(urank)+"\n"+  
      " 关注人数："+str(follow_count)+"\n"+
      " 粉 丝 数："+str(followers_count)+"\n"+
      " 微 博 数："+str(statuses_count)+"\n"+
      " 说　　明："+description+"\n"       
     )

微博昵称：ETF拯救世界
 性　　别：男
 主页地址：https://m.weibo.cn/u/5687069307?uid=5687069307&luicode=10000011&lfid=1005055687069307
 是否认证：是
 等　　级：40
 关注人数：313
 粉 丝 数：317153
 微 博 数：7739
 说　　明：



## 3.查看最近发言
- 查看今天、昨天、本月发言；
- 去除了发言中的连接等无关信息；

### 3.1.今天的发言

In [4]:
from bs4 import BeautifulSoup


#查看最新发言      
def show_weibo_text(user_id,day='今天',url=False,data_root='./'):       
    #从cvs文件恢复数据
    df=read_weibo_data(user_id)
    df['time']=df['created_at']
    df['created_at']=df['created_at'].astype('str').str[0:10]
    if day=='今天' :
        day=pd.datetime.today().strftime('%Y-%m-%d')
        df=df[df['created_at']==day]
    elif day=='昨天':
        day='昨天'
        day=(pd.datetime.today()-timedelta(1)).strftime('%Y-%m-%d')        
        df=df[df['created_at']==day]
    elif day=='本月' :
        day=pd.datetime.today().strftime('%Y-%m')+'-01'   
        df=df[df['created_at']>=day]
    elif day=='上半年' :
        day_start=pd.datetime.today().strftime('%Y')+'-01-01'   
        day_end=pd.datetime.today().strftime('%Y')+'-06-30' 
        df=df[(df['created_at']>=day_start) & (df['created_at']<=day_end)]
    elif day=='下半年' :
        day_start=pd.datetime.today().strftime('%Y')+'-07-01'   
        day_end=pd.datetime.today().strftime('%Y')+'-12-31' 
        df=df[(df['created_at']>=day_start) & (df['created_at']<=day_end)]    
    else:
        return
    df=df.sort_values(['created_at'],ascending=True)
    if len(df)==0:
        print('无内容...')
        return
    for i in range(len(df)): 
        text=df['text'].iloc[i]
        print(df['created_at'].iloc[i]) 
        soup=BeautifulSoup(text,'html.parser')
        print (''.join([item for item in soup.stripped_strings]))
        if url:
            print(str(df['scheme'].iloc[i]))
        print("\r")  

show_weibo_text(user_id,"今天")

2020-01-31
转发微博



### 3.2.昨天的发言

In [5]:
show_weibo_text(user_id,"昨天")

无内容...


### 3.3.本月的发言

In [6]:
show_weibo_text(user_id,"本月")

2020-01-01
新年快乐！！！

2020-01-01
新的一年，早上起来第一件事是锻炼身体这一年，要更加善待自己和他人，要更努力的做事。

2020-01-01
北京5G速度实测

2020-01-02
说一个几周前你会觉得很可笑的事：传媒ETF居然溢价了。市场的羊群疯起来真是不看路的。

2020-01-02
挣点钱而已，意思不大。

2020-01-02
今年是大波动这个判断是不是越来越靠谱了。如果全年大涨，就是又一个短命行情，明年又是万人坑。只有大幅波动才符合长牛慢牛的要求。

2020-01-02
今天押涨有风险吗

2020-01-02
听进去了没有。现在明白了吧。我说的一定要注意啊。

2020-01-02
3000点，账户历史新高了吧。

2020-01-02
春节前如果能维持涨势，春节后又要冲一把。

2020-01-03
喷了。好消息不断。

2020-01-06
黄金原油500传媒环保这些辣鸡这么涨让人感到有点不好意思了。

2020-01-06
500上面第一个压力5650

2020-01-06
意思不大

2020-01-06
投资的时候，舒服重要不重要。谁舒服谁发言。

2020-01-06
再次推荐电影《迷雾》The Mist没看过的一定要看看。看到最后。每次最绝望，最坚持不下去的时候，想想这部电影，会有用。

2020-01-07
早点睡，晚安

2020-01-07
以下几位拿礼物的朋友还没联系我或者小助理，请给我发私信。春节前再不联系就算自动放弃了。ETF张来迟、不忘初心方得始终201820232028、trq81526、丹石_bj

2020-01-07
出门在外，尽量别跟人起冲突。因为你根本想象不到对方有多二。

2020-01-07
传媒上面的压力位在1630。

2020-01-07
传媒辣鸡。你给我赚那么多也是辣鸡。

2020-01-07
你的东西涨了，你心里要清楚是因为价值还是炒作。自己心里得明明白白的。

2020-01-07
回复@兔望雨停:赔了踏踏实实拿10年，赚了烫手拿不了3天。非常经典，绝大多数人都这样。//@兔望雨停:我感觉账户的收益让我觉得好不踏实 只有每天上班的工资让我觉得踏实，明明这几天的收益抵我几个月工资。不知道是怎么回事

2020-01-07
回复@姨夫劝你莫拒绝:拿回现金也不是真的。

### 3.4.上半年的发言

In [7]:
show_weibo_text(user_id,"上半年")

2020-01-01
新年快乐！！！

2020-01-01
新的一年，早上起来第一件事是锻炼身体这一年，要更加善待自己和他人，要更努力的做事。

2020-01-01
北京5G速度实测

2020-01-02
说一个几周前你会觉得很可笑的事：传媒ETF居然溢价了。市场的羊群疯起来真是不看路的。

2020-01-02
挣点钱而已，意思不大。

2020-01-02
今年是大波动这个判断是不是越来越靠谱了。如果全年大涨，就是又一个短命行情，明年又是万人坑。只有大幅波动才符合长牛慢牛的要求。

2020-01-02
今天押涨有风险吗

2020-01-02
听进去了没有。现在明白了吧。我说的一定要注意啊。

2020-01-02
3000点，账户历史新高了吧。

2020-01-02
春节前如果能维持涨势，春节后又要冲一把。

2020-01-03
喷了。好消息不断。

2020-01-06
黄金原油500传媒环保这些辣鸡这么涨让人感到有点不好意思了。

2020-01-06
500上面第一个压力5650

2020-01-06
意思不大

2020-01-06
投资的时候，舒服重要不重要。谁舒服谁发言。

2020-01-06
再次推荐电影《迷雾》The Mist没看过的一定要看看。看到最后。每次最绝望，最坚持不下去的时候，想想这部电影，会有用。

2020-01-07
早点睡，晚安

2020-01-07
以下几位拿礼物的朋友还没联系我或者小助理，请给我发私信。春节前再不联系就算自动放弃了。ETF张来迟、不忘初心方得始终201820232028、trq81526、丹石_bj

2020-01-07
出门在外，尽量别跟人起冲突。因为你根本想象不到对方有多二。

2020-01-07
传媒上面的压力位在1630。

2020-01-07
传媒辣鸡。你给我赚那么多也是辣鸡。

2020-01-07
你的东西涨了，你心里要清楚是因为价值还是炒作。自己心里得明明白白的。

2020-01-07
回复@兔望雨停:赔了踏踏实实拿10年，赚了烫手拿不了3天。非常经典，绝大多数人都这样。//@兔望雨停:我感觉账户的收益让我觉得好不踏实 只有每天上班的工资让我觉得踏实，明明这几天的收益抵我几个月工资。不知道是怎么回事

2020-01-07
回复@姨夫劝你莫拒绝:拿回现金也不是真的。

### 3.5.下半年的发言

In [0]:
show_weibo_text(user_id,"下半年")

## 4.查看原始数据
- 查看DataFrame原始数据。

In [10]:
#查看原始数据    
def show_weibo_data(user_id,data_root='./'):
    #从cvs文件恢复数据
    df=read_weibo_data(user_id)
    df=df.rename(columns={'created_at':'发布时间','text':'内容',
                            'attitudes_count':'点赞','comments_count':'评论','reposts_count':'转发',
                            'scheme':'地址'})
    return df

df=show_weibo_data(user_id)
df

,发布时间,内容,点赞,评论,转发,地址,sended
0,2020-01-31 04:04:23,转发微博,680.0,239.0,41.0,https://m.weibo.cn/status/IrXKfxnVe?mblogid=Ir...,False
1,2020-01-29 18:56:47,回复<a href='/n/老实人的辛酸苦涩'>@老实人的辛酸苦涩</a>:装X就是进化不完...,495.0,88.0,12.0,https://m.weibo.cn/status/IrKJtxHum?mblogid=Ir...,False
2,2020-01-29 18:50:16,回复<a href='/n/沐之以思'>@沐之以思</a>:一部分人没有进化完全而已。身边这...,435.0,51.0,13.0,https://m.weibo.cn/status/IrKGQbujd?mblogid=Ir...,False
3,2020-01-28 23:35:41,"是这样的 <a data-url=""http://t.cn/A6PJiR6H"" href=""...",458.0,190.0,16.0,https://m.weibo.cn/status/IrD8c87Wr?mblogid=Ir...,False
4,2020-01-28 23:29:14,回复<a href='/n/ETF拯救世界'>@ETF拯救世界</a>:又买回来。 <a d...,363.0,85.0,26.0,https://m.weibo.cn/status/IrD5zBWtx?mblogid=Ir...,False
...,...,...,...,...,...,...,...
10427,2015-09-29 13:01:25,我去，居然有大户打赏了！早知道早就来微博了！这么多年少说少赚了100块……我会更加努力的。么...,25.0,33.0,3.0,https://m.weibo.cn/status/CCLD828EG?mblogid=CC...,True
10428,2015-09-29 11:31:02,大概从10年前起，就开始有意识的收集市场极端情况下的新闻报道。真的特别有意思。最近几年，用的...,24.0,14.0,20.0,https://m.weibo.cn/status/CCL2r1DKs?mblogid=CC...,True
10429,2015-09-29 10:26:05,持股的人不要总是嘲笑谩骂持币的投资者。这样一个恒时高估的赌场，大部分时间持币有何过错？在估值...,16.0,14.0,4.0,https://m.weibo.cn/status/CCKC4EnwI?mblogid=CC...,True
10430,2015-09-29 06:30:45,今天的第一个问题是，美帝如此脆弱，我们的73倍pe神创会继续NB吗。第二个问题是，还有两个交...,20.0,27.0,3.0,https://m.weibo.cn/status/CCJ4y5pAY?mblogid=CC...,True


## 5.整理数据
- 获取微博完整信息：完整日期、完整内容。

In [0]:
#整理数据 
def dig_weibo_data(user_id,data_root='./'):
    df=read_weibo_data(user_id)
    #计数器
    count=0   
    #遍历所有数据
    for i in range(len(df)): 
        #开启错误抓取
        try:
            if len(df['created_at'].iloc[i])<=10:
                #计数器累加
                count+=1
                print(' 正在整理：%s 的微博...'%(df['created_at'].iloc[i]),end='\r')
                text,day=get_weibo_text(df['scheme'].iloc[i])
                df['created_at'].iloc[i]=day
                df['text'].iloc[i]=text
        #错误处理
        except Exception as e:
            print(' 错误：%s'%(df['scheme'].iloc[i]),end='\r')
            pass
    #保存数据到cvs 
    save_weibo_data(user_id,df)
    return df

df=dig_weibo_data(id)
df=df[df['created_at'].astype('str').str.len()<=10]   
df
print(len(df))

## 6.生成文本文件
- 生成文本文件，可作为下载后阅读使用；
- 已去除了内容中的多余链接等无关信息。

In [0]:
#生成txt文件      
def write_weibo_text(user_id,data_root='./'):
    #从cvs文件恢复数据
    df=read_weibo_data(user_id)
#     if exists_weibo_file(user_id):  
#         os.remove(file_path)
    for i in range(len(df)): 
        print('-正在整理，第%s条微博...'%(i),end='\r')
        
        text=df['text'].iloc[i]
        soup=BeautifulSoup(text,'html.parser')
        text=''.join([item for item in soup.stripped_strings])
               
        with open(file_path,'a',encoding='utf-8') as fh:                      
            fh.write(
                 "发布时间："+str(df['created_at'].iloc[i])+"\n"
                 +"微博内容："+text+"\n"
                 +"微博地址："+str(df['scheme'].iloc[i])+"\n"
                 +"\n"   
                )

write_weibo_text(user_id,)

## 7.发送邮件
- 发送某个月的所有发言到邮箱；

In [14]:
import sys

#引入自定义库
sys.path.append("../TL")
from tl_mail import *

#发送某月数据
def send_weibo_data(user_id,user_name):
    #检查是否已存在数据
    if not exists_weibo_file(user_id):
        print('未找到 %s 的微博发言数据表！'%user_id)
        return
    #读取数据
    df=read_weibo_data(user_id)
    #计算起止年
    start_year=int(df.created_at.iloc[-1][:4])
    end_year=int(df.created_at.iloc[0][:4])+1
    #月
    m_list=['01','02','03','04','05','06','07','08','09','10','11','12'] 
    #当前月份
    current_month=pd.datetime.today().strftime('%Y-%m')
    #已发送标记
    issend=False
    #遍历年、月
    for y in range(start_year,end_year):
        for m in m_list:
            month=str(y)+'-'+m
            #判断是否为当月，当月不发送
            if current_month==month:
                continue
            #按照年月筛选    
            temp_df=df[(df.created_at.str[0:7]==month) & (df.sended==False)]
            temp_df=temp_df.sort_values('created_at')
            if len(temp_df)==0:
                continue  
            #生成月邮件正文    
            message=''
            for i in range(len(temp_df)):
                #日期
                message+='<p>%s</p>'%temp_df.created_at.iloc[i]  
                #url
                message+='<p>%s</p>'%temp_df.scheme.iloc[i] 
                #正文    
                message+='%s</br></br><hr /></br>'%temp_df.text.iloc[i] 
            #邮件主题    
            subjiect='%s年%s月（微博发言 %s）'%(y,m,user_name)
            #内容标题
            title='<h2>%s</h2></br><hr />'%(subjiect)
            #邮件内容
            message=title+message
            #去除多余的行
            #message=message.replace('<br/><br/>','').replace('</p></br>','</p>').replace('<p><br/></p>','')
            try:
                
                #发送邮件
                send_html_qqmail(subjiect,message)
                #标记已发送
                if len(temp_df)==1:
                    df.sended.iloc[temp_df.index[0]]=True
                else:
                    df.sended.iloc[temp_df.index[-1]:temp_df.index[0]+1]=True
                issend=True    
                print(' %s 的发言发送成功！'%(subjiect),end='\r')
                #延时5秒，防止被踢
                time.sleep(3)
            except Exception as e:
                print(e)
                print(' %s 的发言发送失败！'%(subjiect),end='\r')
    #如果有发送，则保存            
    if issend:
        save_weibo_data(user_id,df)

        
send_weibo_data(user_id,user_name)  

(550, b'Connection frequency limited. http://service.mail.qq.com/cgi-bin/help?subtype=1&&id=20022&&no=1000722 [MD5Wr9Ud9UI8A8CI26leRPiaAm6B2NVXYQEBpP3/GhxiSbUoGyeytUSgys2sMMpsDg== IP: 212.64.102.119]')
 2019年07月（微博发言 ETF拯救世界） 的发言发送失败！(550, b'Connection frequency limited. http://service.mail.qq.com/cgi-bin/help?subtype=1&&id=20022&&no=1000722 [MD5Wr9Ud9UI8A8CI26leRPhOLaFuBBx5hdoTmE7OW3WMAzN+pbuG6pdMhXxjjfmeEg== IP: 212.64.102.119]')
 2019年08月（微博发言 ETF拯救世界） 的发言发送失败！(550, b'Connection frequency limited. http://service.mail.qq.com/cgi-bin/help?subtype=1&&id=20022&&no=1000722 [MD5Wr9Ud9UI8A8CI26leRPi69N8pqaGvdsgp3SmMm4tdk6h7bombjDNTLF0Iy0nY3Q== IP: 212.64.102.119]')
(550, b'Connection frequency limited. http://service.mail.qq.com/cgi-bin/help?subtype=1&&id=20022&&no=1000722 [MD5Wr9Ud9UI8A8CI26leRPiSzMP4hpYu2rDKz/SbwmahnbgMhO860Gsn1bBN3IM4XA== IP: 212.64.102.119]')
 2019年10月（微博发言 ETF拯救世界） 的发言发送失败！(550, b'Connection frequency limited. http://service.mail.qq.com/cgi-bin/help?subtype=1&&id=2002

In [0]:
pd.set_option('display.max_rows',2000)
df=read_weibo_data(user_id)
# del df[df.index==702] 
#del df[(df['created_at']==False)] 
#df.drop(axis=0,labels=['False'],inplace=True) 
#df[(df['created_at']>='2018-09-01') & (df['created_at']<='2018-09-30')] 
#df[(df['created_at']>='2018-09-01') & (df['created_at']<='2018-09-30')].sended=False
#save_weibo_data(user_id,df)
df